# Implementation of the Kuribayashi BERT minus model

## libraries

In [1]:
!pip install transformers --upgrade
!pip install ipywidgets
!pip install IProgress
!pip install datasets
!pip install torch-lr-finder

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
import transformers
from transformers import BertTokenizer, BertConfig
from transformers import BertModel, BertForSequenceClassification
from transformers import BatchEncoding, default_data_collator, DataCollatorWithPadding

import torch
import torch.nn as nn

import numpy as np

from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

import datasets

from torch.utils.data import DataLoader

from tqdm import tqdm

In [3]:
print(transformers.__version__)

4.25.1


## tokenizer

In [4]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

## data

In [5]:
# DATA_FOLDER = '/notebooks/Data/bert_sequence_classification'
DATA_FILE = '/notebooks/KURI-BERT/data/pe_dataset_for_bert_minus.pt'
RESULTS_FOLDER = '/notebooks/KURI-BERT/results'

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
device

device(type='cuda')

## Load data

In [8]:
dataset = torch.load(DATA_FILE)

In [9]:
dataset

DatasetDict({
    train: Dataset({
        features: ['paragraph', 'paragraph_components_list', 'paragraph_labels_list', 'paragraph_markers_list', 'split', 'essay_nr', 'paragraph_labels', 'paragraph_ac_spans', 'paragraph_am_spans'],
        num_rows: 1088
    })
    test: Dataset({
        features: ['paragraph', 'paragraph_components_list', 'paragraph_labels_list', 'paragraph_markers_list', 'split', 'essay_nr', 'paragraph_labels', 'paragraph_ac_spans', 'paragraph_am_spans'],
        num_rows: 358
    })
    validation: Dataset({
        features: ['paragraph', 'paragraph_components_list', 'paragraph_labels_list', 'paragraph_markers_list', 'split', 'essay_nr', 'paragraph_labels', 'paragraph_ac_spans', 'paragraph_am_spans'],
        num_rows: 273
    })
})

### preprocessing

In [10]:
MAX_LENGTH = 0

for split in ['train', 'test', 'validation']:
    
    for col_name in ['paragraph_am_spans', 'paragraph_ac_spans']:
        
        for x in dataset[split][col_name]:
        
            if len(x) > MAX_LENGTH:
                
                MAX_LENGTH = len(x)

In [11]:
def get_padding(batch, padding_target):    
    
    if padding_target == 'am_spans':
        
        col_name = 'paragraph_am_spans'
        padding_val = [[-1,-1]]
        max_length = MAX_LENGTH
        
    elif padding_target == 'ac_spans':
        
        col_name = 'paragraph_ac_spans'
        padding_val = [[-1,-1]]
        max_length = MAX_LENGTH
        
    elif padding_target == 'label':
    
        col_name = 'paragraph_labels'
        padding_val = [-100] # -1 previously       
        max_length = MAX_LENGTH # max([len(l) for l in batch[col_name]]) # cause some batch had 4 x 10

    padded_spans = []

    for idx, span in enumerate(batch[col_name]):

        padded_span = batch[col_name][idx] + (max_length - len(span)) * padding_val
        padded_spans.append(padded_span)

    return padded_spans         

In [12]:
def get_combined_spans(am_spans_ll, ac_spans_ll):
    
    spans_ll = []
    
    for am_spans, ac_spans in zip(am_spans_ll, ac_spans_ll):
        
        spans = []
        
        for am_span, ac_span in zip(am_spans, ac_spans):

            span = [am_span, ac_span]
            spans.extend(span)
            
        spans_ll.append(spans)

    return spans_ll

### tokenize 

In [13]:
# max_length = 200 for use in the max_length in the tokenizer so that the things are of equal dim.

In [14]:
def tokenize(batch):
    
    tokenized_text = tokenizer(batch['paragraph'], truncation=True, padding=True, max_length=512)
    tokenized_text['label'] = get_padding(batch, 'label')
    tokenized_text['am_spans'] = get_padding(batch, 'am_spans')
    tokenized_text['ac_spans'] = get_padding(batch, 'ac_spans')
    tokenized_text['spans'] = get_combined_spans(tokenized_text['am_spans'], tokenized_text['ac_spans'])      
    
    return tokenized_text

In [15]:
dataset = dataset.map(tokenize, batched=True, batch_size=len(dataset['train']))

Parameter 'function'=<function tokenize at 0x7f1adb7e0e50> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [16]:
dataset

DatasetDict({
    train: Dataset({
        features: ['ac_spans', 'am_spans', 'attention_mask', 'essay_nr', 'input_ids', 'label', 'paragraph', 'paragraph_ac_spans', 'paragraph_am_spans', 'paragraph_components_list', 'paragraph_labels', 'paragraph_labels_list', 'paragraph_markers_list', 'spans', 'split', 'token_type_ids'],
        num_rows: 1088
    })
    test: Dataset({
        features: ['ac_spans', 'am_spans', 'attention_mask', 'essay_nr', 'input_ids', 'label', 'paragraph', 'paragraph_ac_spans', 'paragraph_am_spans', 'paragraph_components_list', 'paragraph_labels', 'paragraph_labels_list', 'paragraph_markers_list', 'spans', 'split', 'token_type_ids'],
        num_rows: 358
    })
    validation: Dataset({
        features: ['ac_spans', 'am_spans', 'attention_mask', 'essay_nr', 'input_ids', 'label', 'paragraph', 'paragraph_ac_spans', 'paragraph_am_spans', 'paragraph_components_list', 'paragraph_labels', 'paragraph_labels_list', 'paragraph_markers_list', 'spans', 'split', 'token_type_

In [17]:
dataset['train'][0]

{'ac_spans': [[2, 21],
  [27, 43],
  [45, 80],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1]],
 'am_spans': [[0, 0],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1]],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
 

In [18]:
dataset['train'].features

{'ac_spans': Sequence(feature=Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), length=-1, id=None),
 'am_spans': Sequence(feature=Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'essay_nr': Value(dtype='string', id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'label': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
 'paragraph': Value(dtype='string', id=None),
 'paragraph_ac_spans': Sequence(feature=Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), length=-1, id=None),
 'paragraph_am_spans': Sequence(feature=Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), length=-1, id=None),
 'paragraph_components_list': Sequence(feature=Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), length=-1, id=None),
 'paragraph_labels': S

In [19]:
dataset['test'].features

{'ac_spans': Sequence(feature=Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), length=-1, id=None),
 'am_spans': Sequence(feature=Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'essay_nr': Value(dtype='string', id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'label': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
 'paragraph': Value(dtype='string', id=None),
 'paragraph_ac_spans': Sequence(feature=Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), length=-1, id=None),
 'paragraph_am_spans': Sequence(feature=Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), length=-1, id=None),
 'paragraph_components_list': Sequence(feature=Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), length=-1, id=None),
 'paragraph_labels': S

In [20]:
dataset['test'].features['spans'] = datasets.Array2D(shape=(24, 2), dtype="int32")
dataset['train'].features['spans'] = datasets.Array2D(shape=(24, 2), dtype="int32")
dataset['validation'].features['spans'] = datasets.Array2D(shape=(24, 2), dtype="int32")

In [21]:
dataset = dataset.map(lambda batch: batch, batched=True)

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [22]:
dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'token_type_ids', 'spans', 'label'])

## span representation function

In [23]:
def get_span_representations(outputs, spans):

    batch_size = spans.shape[0]
    nr_span_indices = spans.shape[1]
    
    idx_l_ams = range(0, nr_span_indices, 2) # [0,2,4,6 etc]
    idx_l_acs = range(1, nr_span_indices, 2) # [1,3,5,7 etc]
    
    am_spans = spans[:, idx_l_ams, :] + 1 # adds 1 to all span indices (both am and ac) to offset for the CLS token in the input_ids.
    ac_spans = spans[:, idx_l_acs, :] + 1
    
    am_spans = am_spans.flatten(start_dim=1)
    ac_spans = ac_spans.flatten(start_dim=1)
    
    #print("am spans:", am_spans.shape)
    #print("ac spans:", ac_spans.shape)
    
    outputs_am = outputs[:,am_spans,:]
    outputs_am = torch.cat([outputs_am[i,i,:,:] for i in range(batch_size)], dim=0)
    outputs_am = outputs_am.reshape(batch_size, nr_span_indices, -1)
    
    #print("outputs am:", outputs_am.shape)
    
    outputs_am_r = torch.cat([outputs_am[:,i,:] - outputs_am[:,i+1,:] for i in range(0, nr_span_indices, 2)], dim=1)
    outputs_am_r = outputs_am_r.reshape(batch_size, -1, 768)
    
    outputs_am_l = torch.cat([outputs_am[:,i+1,:] - outputs_am[:,i,:] for i in range(0, nr_span_indices, 2)], dim=1)
    outputs_am_l = outputs_am_l.reshape(batch_size, -1, 768)
    
    am_minus_representations = torch.cat([outputs_am_r, outputs_am_l], dim=-1)
    
    #print("am_minus_representations:", am_minus_representations.shape)
    
    outputs_ac = outputs[:,ac_spans,:]
    outputs_ac = torch.cat([outputs_ac[i,i,:,:] for i in range(batch_size)], dim=0)
    outputs_ac = outputs_ac.reshape(batch_size, nr_span_indices, -1)
    
    #print("outputs ac:", outputs_ac.shape)
    
    outputs_ac_r = torch.cat([outputs_ac[:,i,:] - outputs_ac[:,i+1,:] for i in range(0, nr_span_indices, 2)], dim=1)
    outputs_ac_r = outputs_ac_r.reshape(batch_size, -1, 768)
    
    outputs_ac_l = torch.cat([outputs_ac[:,i+1,:] - outputs_ac[:,i,:] for i in range(0, nr_span_indices, 2)], dim=1)
    outputs_ac_l = outputs_ac_l.reshape(batch_size, -1, 768)
    
    ac_minus_representations = torch.cat([outputs_ac_r, outputs_ac_l], dim=-1)
    
    #print("ac_minus_representations:", ac_minus_representations.shape)
    
    return am_minus_representations, ac_minus_representations                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

## custom BERT model

In [24]:
class CustomBERTKuri(nn.Module):

    def __init__(self, first_model, model_am, model_ac, nr_classes):
        
        super(CustomBERTKuri, self).__init__()
        
        self.first_model = first_model
        
        self.intermediate_linear_am = nn.Linear(1536, 768)
        self.intermediate_linear_ac = nn.Linear(1536, 768)        
        
        self.model_am = model_am
        self.model_ac = model_ac
        
        self.nr_classes = nr_classes
                
        self.fc = nn.Linear(self.model_am.config.hidden_size + self.model_ac.config.hidden_size, self.nr_classes)        

    def forward(self, inputs):
        
        batch_tokenized, batch_spans = inputs 
        outputs = self.first_model(batch_tokenized)[0] # ** removed to correct error.
        # spans = batch_spans # remove this spans thing cause we are now giving it just the spans themselves.
        am_minus_representations, ac_minus_representations = get_span_representations(outputs, batch_spans)

        am_minus_representations = self.intermediate_linear_am(am_minus_representations)
        ac_minus_representations = self.intermediate_linear_ac(ac_minus_representations)

        output_model_am = self.model_am(inputs_embeds = am_minus_representations)[0]
        output_model_ac = self.model_ac(inputs_embeds = ac_minus_representations)[0]

        adu_representations = torch.cat([output_model_am, output_model_ac], dim=-1)
        # print("adu rep:", adu_representations.shape)
        output = self.fc(adu_representations)
        # print("model class output avant reshape:", output.shape)
        # output = output.reshape(-1, self.nr_classes)
        # print("model class output apres:", output.shape)
        return output

## Run

In [25]:
NB_EPOCHS = 40
BATCH_SIZE = 48

In [26]:
first_model = BertModel(BertConfig.from_pretrained("bert-base-uncased"))

In [27]:
second_model = BertModel(BertConfig.from_pretrained("bert-base-uncased"))

In [28]:
#model_am = BertModel(BertConfig.from_pretrained("bert-base-uncased"))

In [29]:
#model_ac = BertModel(BertConfig.from_pretrained("bert-base-uncased"))

In [30]:
custom_model = CustomBERTKuri(first_model, second_model, second_model, 3)

In [31]:
custom_model.to(device)

CustomBERTKuri(
  (first_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_aff

In [32]:
loss = nn.CrossEntropyLoss(ignore_index=- 100)

In [33]:
optimizer = torch.optim.AdamW(custom_model.parameters(), lr=1.2328467394420658e-05) # best lr for two same models
# different ones best lR: 1.8738174228603844e-05
# same ones best LR: 2.3101297000831607e-05

In [34]:
# 1.8738174228603844e-05
# best learning rate found by the whole leslie business

In [35]:
NR_BATCHES = len(dataset['train']) / BATCH_SIZE
num_training_steps = NB_EPOCHS * NR_BATCHES
num_warmup_steps = int(0.2 * num_training_steps)

In [36]:
def lr_lambda(current_step: int):
        if current_step < num_warmup_steps:
            return float(current_step) / float(max(1, num_warmup_steps))
        return max(
            0.0, float(num_training_steps - current_step) / float(max(1, num_training_steps - num_warmup_steps))
        )

    # return LambdaLR(optimizer, lr_lambda, last_epoch)

In [37]:
# commented for LR Finder. remove it from optimizer.
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lr_lambda)

### create dataloaders

In [38]:
train_dataloader = DataLoader(dataset['train'], batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(dataset['validation'], batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(dataset['test'], batch_size=BATCH_SIZE, shuffle=True)

In [39]:
def flatten_list(list_of_lists):
    return [x for sublist in list_of_lists for x in sublist]

In [40]:
def remove_dummy_labels(test_preds, test_labels):
    
    idxes = []
    test_labels_l = []
    for idx, val in enumerate(test_labels):
        if val != -100:
            idxes.append(idx)
            test_labels_l.append(val)
    
    test_preds_l = []
    for idx, val in enumerate(test_preds):
        for good_idx in idxes:
            if idx == good_idx:
                test_preds_l.append(val)
        
    return test_preds_l, test_labels_l

### training loop

In [41]:
from transformers import Trainer, TrainingArguments
from datasets import load_metric

In [42]:
import random
from torch_lr_finder import LRFinder

In [43]:
# def compute_loss(self, model, inputs, return_outputs=False):

#     labels = inputs["labels"]
#     labels = labels.flatten() # xxx.



#     outputs = model(inputs['input_ids'], inputs['spans'])
#     outputs = outputs.flatten(0,1) # xxx. for the 4 x 12 , 3 output of the main class.


#     loss_fct = nn.CrossEntropyLoss()#(weight=class_weights)
#     # loss = loss_fct(outputs, labels.flatten())
#     loss = loss_fct(outputs, labels) # xxx
#     #print("loss:", loss)
#     return (loss, outputs) if return_outputs else loss

In [44]:
# metric = load_metric('f1')

# def compute_metrics(eval_pred):

#     logits, labels = eval_pred

    
#     print("logits", logits.shape)    
#     predictions = np.argmax(logits, axis=-1)
    
#     print("preds:", predictions.shape)
    
#     return metric.compute(predictions=predictions, references=labels, average='macro')

### LR Finder Leslie Smith 

## training 

In [45]:
def train(model, loss=None, optimizer=None, train_dataloader=None, val_dataloader=None, nb_epochs=20):
    """Training loop"""

    min_f1 = -torch.inf
    train_losses = []
    val_losses = []

    # Iterrate over epochs
    for e in range(nb_epochs):

        # Training
        train_loss = 0.0

        for batch in tqdm(train_dataloader):            
            
            # unpack batch             
            labels = batch['label'].to(device)
            spans = batch['spans'].to(device)
            input_ids = batch['input_ids'].to(device)
            
            inputs = input_ids, spans
            
            # Reset gradients to 0
            optimizer.zero_grad()

            # Forward Pass
            outputs = model(inputs)
            
            # Compute training loss
            current_loss = loss(outputs.flatten(0,1), labels.flatten())
            train_loss += current_loss.detach().item()

            # Compute gradients
            current_loss.backward()

            # Update weights
            optimizer.step()            
            
            del batch
        
        scheduler.step()
            
        
        # Validation
        val_loss = 0.0

        # Put model in eval mode
        model.eval()
        
        preds_l = []
        labels_l = []
        
        for batch in tqdm(val_dataloader):            
            
            # unpack batch             
            labels = batch['label'].to(device)
            spans = batch['spans'].to(device)
            input_ids = batch['input_ids'].to(device)
            
            inputs = input_ids, spans
            
            # Forward Pass
            outputs = model(inputs)

            # Compute validation loss
            current_loss = loss(outputs.flatten(0,1), labels.flatten())
            val_loss += current_loss.detach().item()
            
            preds_for_f1 = torch.argmax(outputs, dim=2).flatten().tolist()
            labels_for_f1 = labels.flatten().tolist()
            
            preds_l.append(preds_for_f1)
            labels_l.append(labels_for_f1)
            
            del batch
        
        # Prints
        
        preds_l = flatten_list(preds_l)
        labels_l = flatten_list(labels_l)
        
        preds_l, labels_l = remove_dummy_labels(preds_l, labels_l)
        
        f1_score_epoch = f1_score(preds_l, labels_l, average='macro')        
        
        print(f"Epoch {e+1}/{nb_epochs} \
                \t Training Loss: {train_loss/len(train_dataloader):.3f} \
                \t Validation Loss: {val_loss/len(val_dataloader):.3f} \
                \t F1 score: {f1_score_epoch}")
        
        train_losses.append(train_loss/len(train_dataloader))
        val_losses.append(val_loss/len(val_dataloader))
        

        # Save model if val loss decreases
        if f1_score_epoch > min_f1:

            min_f1 = f1_score_epoch
            torch.save(model.first_model.state_dict(), 'first_model.pt')
            torch.save(model.model_am.state_dict(), 'second_model.pt')
            #torch.save(model.model_ac.state_dict(), 'model_ac.pt')
            torch.save(model.state_dict(), 'best_model.pt')
            
    return train_losses, val_losses

In [46]:
train_losses, val_losses = train(custom_model, loss, optimizer, train_dataloader, val_dataloader, NB_EPOCHS)

100%|██████████| 6/6 [00:02<00:00,  2.66it/s]


Epoch 1/40                 	 Training Loss: 1.311                 	 Validation Loss: 1.307                 	 F1 score: 0.1510444377736603


100%|██████████| 6/6 [00:02<00:00,  2.70it/s]


Epoch 2/40                 	 Training Loss: 1.218                 	 Validation Loss: 1.133                 	 F1 score: 0.27257187922777404


100%|██████████| 6/6 [00:02<00:00,  2.73it/s]


...

Epoch 38/40                 	 Training Loss: 0.034                 	 Validation Loss: 0.848                 	 F1 score: 0.6723759915747126


100%|██████████| 6/6 [00:02<00:00,  2.68it/s]


Epoch 39/40                 	 Training Loss: 0.028                 	 Validation Loss: 0.855                 	 F1 score: 0.6759633635660962


100%|██████████| 6/6 [00:02<00:00,  2.68it/s]

Epoch 40/40                 	 Training Loss: 0.022                 	 Validation Loss: 0.905                 	 F1 score: 0.6725091772384112


### Predictions

In [47]:
# # Load best model
# network_2 = Network()

# network_2.load_state_dict(torch.load(path))
# network_2.eval()

In [48]:
first_model = BertModel(BertConfig.from_pretrained("bert-base-uncased"))
first_model.load_state_dict(torch.load('first_model.pt'))

<All keys matched successfully>

In [49]:
second_model = BertModel(BertConfig.from_pretrained("bert-base-uncased"))
second_model.load_state_dict(torch.load('second_model.pt'))

<All keys matched successfully>

In [50]:
# model_ac = BertModel(BertConfig.from_pretrained("bert-base-uncased"))
# model_ac.load_state_dict(torch.load('model_ac.pt'))

In [51]:
# Load best model

custom_model_2 = CustomBERTKuri(first_model, second_model, second_model, 3)
custom_model_2.load_state_dict(torch.load('best_model.pt'))

custom_model_2.to(device).eval()

CustomBERTKuri(
  (first_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_aff

In [52]:
def predict(model, test_dataloader=None):
    
    """Prediction loop"""

    preds_l = []
    labels_l = []
    
    model.eval()

    for batch in test_dataloader:            
            
        # unpack batch             
        labels = batch['label'].to(device).flatten().tolist()
        spans = batch['spans'].to(device)
        input_ids = batch['input_ids'].to(device)
        
        inputs = input_ids, spans

        # get output
        
        raw_preds = model(inputs).to('cpu')
        # print(raw_preds.shape)
        raw_preds = raw_preds.detach()#.numpy()

        # Compute argmax
        
        predictions = torch.argmax(raw_preds, dim=2).flatten().tolist()
        preds_l.append(predictions)
        labels_l.append(labels)        
        
        del batch
            
    return flatten_list(preds_l), flatten_list(labels_l)

In [53]:
#test_preds, test_labels = predict(custom_model, test_dataloader)

In [54]:
#print(classification_report(test_labels, test_preds, digits=3))

In [55]:
# remove -100s

In [56]:
#test_preds_l, test_labels_l = remove_dummy_labels(test_preds, test_labels)

In [57]:
#print(classification_report(test_labels_l, test_preds_l, digits=3))

In [58]:
### -100 done!

In [59]:
test_preds, test_labels = predict(custom_model_2, test_dataloader)

In [60]:
print(classification_report(test_labels, test_preds, digits=3))

              precision    recall  f1-score   support

        -100      0.000     0.000     0.000      3033
           0      0.140     0.735     0.235       155
           1      0.128     0.472     0.202       303
           2      0.308     0.906     0.460       805

    accuracy                          0.230      4296
   macro avg      0.144     0.528     0.224      4296
weighted avg      0.072     0.230     0.109      4296



/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [61]:
test_preds_l, test_labels_l = remove_dummy_labels(test_preds, test_labels)

In [62]:
print(classification_report(test_labels_l, test_preds_l, digits=3))

              precision    recall  f1-score   support

           0      0.786     0.735     0.760       155
           1      0.598     0.472     0.528       303
           2      0.829     0.906     0.866       805

    accuracy                          0.781      1263
   macro avg      0.738     0.704     0.718      1263
weighted avg      0.769     0.781     0.772      1263

